In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

from pcp import pcp
from pcp import radar_pcp
from skimage.io import imread

In [ ]:
image_dir = 'RGB_images'
bounds_file_path = 'bounds.txt'
with open(bounds_file_path, "rb") as fp:   # Unpickling
    bounds = pickle.load(fp)
images = np.array([imread(os.path.join(image_dir, 'rgb_{}.png'.format(i)), 'gray') for i in range(len(bounds))])

In [ ]:
D = images.reshape(images.shape[0], -1).T

def build_M_F(shape, bounds, gamma_b, gamma_f):
    M_F = np.zeros(shape)
    for i in range(len(bounds)):
        for j in range(len(bounds[i])):
            f_l = bounds[i][j, 0]
            f_r = bounds[i][j, 1]
            M_F[i, f_l:f_r] = gamma_f
    M_F[M_F != gamma_f] = gamma_b
    return M_F.reshape(shape[0], -1).T

In [ ]:
gamma_b = 1.25
gamma_f = 0.75
M_F = build_M_F(images.shape, bounds, gamma_b, gamma_f)

In [ ]:
L_pcp, S_pcp, _ = pcp(D, verbose=True, maxiter=500)
L_radar, S_radar, _ = radar_pcp(D, M_F, verbose=True, maxiter=500)

In [ ]:
i = 0
plt.figure(figsize=(10,6))
plt.title('Original')
plt.imshow(images[i], 'gray')
plt.axis('off')
plt.figure(figsize=(15,15))
plt.subplot(321)
plt.title('L: PCP')
plt.imshow(L_pcp[:, i].reshape(images.shape[1], images.shape[2]), 'gray')
plt.axis('off')
plt.subplot(322)
plt.title('abs(S): PCP')
plt.imshow(abs(S_pcp[:, i].reshape(images.shape[1], images.shape[2])), 'gray')
plt.axis('off')
plt.subplot(323)
plt.title('L: Radar-PCP')
plt.imshow(L_radar[:, i].reshape(images.shape[1], images.shape[2]), 'gray')
plt.axis('off')
plt.subplot(324)
plt.title('abs(S): Radar-PCP')
plt.imshow(abs(S_radar[:, i].reshape(images.shape[1], images.shape[2])), 'gray')
plt.axis('off')
plt.subplot(325)
plt.title('Absolute BG Difference')
plt.imshow(abs(L_radar[:, i]-L_pcp[:, i]).reshape(images.shape[1], images.shape[2]), 'hot')
plt.colorbar()
plt.axis('off')
plt.subplot(326)
plt.title('Absolute FG Difference')
plt.imshow(abs(S_radar[:, i]-S_pcp[:, i]).reshape(images.shape[1], images.shape[2]), 'hot')
plt.colorbar()
plt.axis('off')